In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape
from keras import layers
#from keras.layers import BatchNormalization
from keras.models import Model
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import  PCA
from mpl_toolkits.mplot3d import Axes3D
import IPython
import librosa.display
import librosa
import pickle

In [2]:
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.9.0


In [35]:
optimizer = tf.keras.optimizers.Adam(0.0001)
latent_dim = 128
sr = 48000
fx = ["Clean", "TubeScreamer", "BluesDriver", "RAT", "Chorus", "Phaser", "Flanger", "Tape Echo","Sweep Echo", "Digital Delay", "Hall Reverb", "PlateReverb", "Spring Reverb"]


In [46]:
with open('/Users/mezaga93/Desktop/sintesis_aciml.pkl', 'rb') as f:
  waveforms = pickle.load(f)
print(waveforms.shape)

(5000, 192000)


In [49]:
waveforms = waveforms[:600, :48000]
print(waveforms.shape)

(600, 48000)


In [23]:
waveforms = waveforms[:850]

In [50]:
###normalizar y cambio de dimensionalidad
mu_wave = np.mean(waveforms, axis= 1)[...,np.newaxis]
max_wave = np.max(waveforms, axis=1)[...,np.newaxis]

Xw = (waveforms- mu_wave)/max_wave
print(Xw.shape)

(600, 48000)


In [51]:
#####reshape de los datos
Xw = Xw[...,np.newaxis][...,np.newaxis]
print(Xw.shape)

(600, 48000, 1, 1)


In [81]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    #@tf.function
    def call(self, inputs):
        z_mean, z_log_var = inputs
        #z_mean = np.array([input[0]])
        #z_log_var = np.array([input[1]])
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [82]:
class KLDivergenceRegularizer(tf.keras.regularizers.Regularizer):
  def __init__(self, iters: tf.Variable, warm_up_iters: int, latent_size: int):
    self._iters = np.array([iters])
    self._warm_up_iters = np.array([warm_up_iters])
    self.latent_size = latent_size
  
  @tf.function
  def __call__(self, activation):
    # note: activity regularizers automatically divide by batch size
    mu= activation[:self.latent_size]
    log_var = activation[self.latent_size:]
    k = np.min(self._iters / self._warm_up_iters)
    return -0.5 * k * K.sum(1+ log_var - K.square(mu) - K.exp(log_var))

In [83]:
warm_up_iters = 1000 
optimizer = tf.keras.optimizers.Adam(0.001)

In [84]:
latent_dim = 256
original_dim = (48000, 1, 1)
encoder_inputs = keras.Input(original_dim)                                                   # bat, samps, ?, ch
x = layers.Conv2D(64, (93,1), activation="relu", padding="same",strides=(4,1))(encoder_inputs) # [69, 48000, 1, 1] -> [69, 12000, 1, 64]
x = layers.Conv2D(128,(93,1), activation="relu", padding="same",strides=(4,1))(x)
x = layers.Conv2D(256,(93,1), activation="relu", padding="same",strides=(4,1))(x)
latent = layers.Conv2D(512,(93,1), activation="relu", padding="same",strides=(3,1))(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
z_mean = layers.Dense(units=latent_dim, name='Z-Mean')(x) # Mean component
z_log_sigma = layers.Dense(units=latent_dim, name='Z-Log-Sigma')(x) # Standard deviation component
#######que pasa con z_regular nunca se usa?
z_regular = tf.keras.layers.Concatenate(activity_regularizer= KLDivergenceRegularizer(optimizer.iterations, warm_up_iters,latent_dim))([z_mean,z_log_sigma])
z = Sampling()([z_mean, z_log_sigma])
#z = Sampling()(z_regular)
vae_encoder = keras.Model(encoder_inputs, [z_mean, z_log_sigma, z], name="encoder_vae")
vae_encoder.summary()

Model: "encoder_vae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 48000, 1, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 12000, 1, 64  6016        ['input_11[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_21 (Conv2D)             (None, 3000, 1, 128  761984      ['conv2d_20[0][0]']              
                                )                                                       

In [85]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(750 * 1 * 256, activation="relu")(latent_inputs)
x = layers.Reshape((750, 1, 256))(x)
#x = layers.Conv2DTranspose(512,(4,1), activation="relu",padding= "same",strides= (3,1))(x)
x = layers.Conv2DTranspose(128,(93,1), activation="relu",padding= "same",strides= (4,1))(x)
x = layers.Conv2DTranspose(64,(93,1), activation="relu",padding= "same",strides= (4,1))(x)
#x = layers.Conv2DTranspose(64,(16,1), activation="relu",padding= "same",strides=(4,1))(x)
decoder_outputs = layers.Conv2DTranspose(1, (93, 1), activation="tanh", padding="same", strides = (4,1))(x)
vae_decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder_vae")
vae_decoder.summary()

Model: "decoder_vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 256)]             0         
                                                                 
 dense_11 (Dense)            (None, 192000)            49344000  
                                                                 
 reshape_5 (Reshape)         (None, 750, 1, 256)       0         
                                                                 
 conv2d_transpose_15 (Conv2D  (None, 3000, 1, 128)     3047552   
 Transpose)                                                      
                                                                 
 conv2d_transpose_16 (Conv2D  (None, 12000, 1, 64)     761920    
 Transpose)                                                      
                                                                 
 conv2d_transpose_17 (Conv2D  (None, 48000, 1, 1)      

In [86]:
# Define outputs from a VAE model by specifying how the encoder-decoder models are linked
outpt = vae_decoder(vae_encoder(encoder_inputs)[2]) # note, outputs available from encoder model are z_mean, z_log_sigma and z. We take z by specifying [2]
# Instantiate a VAE model
vae = Model(inputs=encoder_inputs, outputs=outpt, name='VAE-Model')
vae.compile(optimizer= optimizer,loss = 'mean_squared_error')

#vae.summary()

In [87]:
tr_logs = vae.fit(Xw, Xw, shuffle = True, batch_size= 5, epochs= 100)

Epoch 1/100


2022-06-14 20:51:24.065025: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - 34s 278ms/step - loss: 0.0898
Epoch 2/100
120/120 [==============================] - 34s 279ms/step - loss: 0.0898
Epoch 3/100
120/120 [==============================] - 34s 281ms/step - loss: 0.0898
Epoch 4/100
120/120 [==============================] - 34s 286ms/step - loss: 0.0898
Epoch 5/100
120/120 [==============================] - 34s 280ms/step - loss: 0.0898
Epoch 6/100
 11/120 [=>............................] - ETA: 30s - loss: 0.0822

KeyboardInterrupt: 